In [2]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

from google.colab import drive
drive.mount('/content/drive')

class MLP(nn.Module):
  def __init__(self, n_units, init_scale=1.0):
    super(MLP, self).__init__()

    self._n_units = copy.copy(n_units)
    self._layers = []
    for i in range(1, len(n_units)):
      layer = nn.Linear(n_units[i-1], n_units[i], bias=False)
      variance = math.sqrt(2.0 / (n_units[i-1] + n_units[i]))
      layer.weight.data.normal_(0.0, init_scale * variance)
      self._layers.append(layer)

      name = 'fc%d' % i
      if i == len(n_units) - 1:
        name = 'fc'  # the prediction layer is just called fc
      self.add_module(name, layer)


  def forward(self, x):
    x = x.view(-1, self._n_units[0])
    out = self._layers[0](x)
    for layer in self._layers[1:]:
      out = F.relu(out)
      out = layer(out)
    return out



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/'My Drive'

/content/drive/My Drive


In [0]:
def get_data(seed):
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)
  np.random.seed(seed)

  probability_of_random = 1.0
  labels = np.array(train_dataset.targets) 
  mask = np.random.rand(len(labels)) <= probability_of_random #create mask of length labels, where entries drawn from [0,1].
  rnd_labels = np.random.choice(10, mask.sum())               #create random labels 1-10 of length of mask
  labels[mask] = rnd_labels
  labels = [int(x) for x in labels]
  train_dataset.targets = labels                              #assign new random labels to dataset
  
  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=128,
                          num_workers=4,
                          shuffle=True)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=128,
                         num_workers=4,
                         shuffle=False)
  return train_loader, test_loader

In [0]:
def compute_acc(model, data_loader):
    correct_pred, num_examples = 0, 0
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
        probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        assert predicted_labels.size() == targets.size()
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [7]:
weights = []
models = []

for i in range(10):
  model = torch.load('MLP_randomlabels_seed' + str(i))
  train_loader, test_loader = get_data(i)
  
  models.append(model)
  weights.append(model.fc1.weight)

  train_acc = compute_acc(model, train_loader)        
  print(f'train ACC: {train_acc:.2f}, seed: {i}')
        
  test_acc = compute_acc(model, test_loader)        
  print(f'Test ACC: {test_acc:.2f}')

Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.28, seed: 0
Test ACC: 9.79
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.30, seed: 1
Test ACC: 9.83
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.25, seed: 2
Test ACC: 10.42
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.22, seed: 3
Test ACC: 10.28
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.30, seed: 4
Test ACC: 10.17
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.28, seed: 5
Test ACC: 10.72
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.28, seed: 6
Test ACC: 9.73
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.27, seed: 7
Test ACC: 10.01
Files already downloaded and verified
Files already downloa

In [0]:
norms = []
for i in range(10):
  for j in range(10):
    if i != j:
      cross_corr = weights[i] @ weights[j].t()
      norms.append(torch.norm(cross_corr, p='fro'))


In [12]:
print(norms)

[tensor(131.7955, grad_fn=<NormBackward0>), tensor(131.3481, grad_fn=<NormBackward0>), tensor(131.5771, grad_fn=<NormBackward0>), tensor(132.1409, grad_fn=<NormBackward0>), tensor(132.0454, grad_fn=<NormBackward0>), tensor(132.0513, grad_fn=<NormBackward0>), tensor(131.6216, grad_fn=<NormBackward0>), tensor(131.2394, grad_fn=<NormBackward0>), tensor(131.4613, grad_fn=<NormBackward0>), tensor(131.7958, grad_fn=<NormBackward0>), tensor(132.0717, grad_fn=<NormBackward0>), tensor(132.3100, grad_fn=<NormBackward0>), tensor(132.5443, grad_fn=<NormBackward0>), tensor(132.4661, grad_fn=<NormBackward0>), tensor(132.5544, grad_fn=<NormBackward0>), tensor(132.2428, grad_fn=<NormBackward0>), tensor(131.3037, grad_fn=<NormBackward0>), tensor(131.7107, grad_fn=<NormBackward0>), tensor(131.3471, grad_fn=<NormBackward0>), tensor(132.0721, grad_fn=<NormBackward0>), tensor(132.0779, grad_fn=<NormBackward0>), tensor(132.7404, grad_fn=<NormBackward0>), tensor(132.3577, grad_fn=<NormBackward0>), tensor(132